In [ ]:
!pip install -U transformers==3.0.0

In [ ]:
!python -m nltk.downloader punkt

/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
!git clone https://github.com/patil-suraj/question_generation.git

fatal: destination path 'question_generation' already exists and is not an empty directory.


In [ ]:
import pandas as pd
import random
import re 
import csv
import urllib.request as req
import requests
from bs4 import BeautifulSoup
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir("/content/drive/MyDrive/END-3")

In [ ]:
df = pd.read_excel('urlallocation.xlsx')
teamurllist = []
df['url']=df['url'].str.strip()
for index, row in df.iterrows():
    #print (row["url"])
    teamurllist.append(row["url"])
teamurllist()

SyntaxError: ignored

In [ ]:
len(df['url'])

60

In [ ]:
start_idx = 0
end_idx = 1
#end_idx = len(df['url'])

In [ ]:

uniqdict = {}
qadict = {}

#nlp = pipeline("question-generation", model="valhalla/t5-base-qg-hl")
lengthlist = [512 , 1024, 2048 , 3072 , 4096]


for url in teamurllist[start_idx:end_idx]:
  #url = 'https://pytorch.org/docs/stable/tensor_attributes.html#type-promotion-doc' 
  # 'https://pytorch.org/docs/stable/tensor_attributes.html#type-promotion-doc'
  #'https://pytorch.org/docs/stable/autograd.html'
  # 'https://pytorch.org/docs/stable/generated/torch.load.html#torch.load'
  print(url)
  try:
    html = req.urlopen(url).read()
  except:
    continue
  soup = BeautifulSoup(html)




  article = soup.find('article')
  
  if article:

    sctns = article.find_all('div', class_="section")

    for sctn in sctns:
      ptags = sctn.find_all('p')

      
      ptextlist = []

      i = 1
      for ptag in ptags:
        ptextlist.append([ i , ptag.get_text(strip=True)])
        i += 1

      #print(ptextlist)

      ptextlistlen = len(ptextlist)

      ptextlt1024list = []
      alltextdone = False

      for j in range(40):
        ptext = ""
        if not alltextdone:

          for k in range(j,ptextlistlen):
          
            arry = ptextlist[k]
            #print(arry)
            if len(ptext) + len(arry[1]) + 1 < 512: #1024:
              ptext += arry[1]
              ptext += ' '

              if arry[0] == ptextlistlen:

                #print('len 1',len(ptext))
                try:
                  listofdict = nlp(ptext)

                  for qas in listofdict:
                    qas['context'] = ptext
                    qas['source'] = url
                    qadict[qas['question']] = qas

                except:
                  1 == 1
                
                
                try:
                  listofdict = nlp_mtask(ptext)

                  for qas in listofdict:
                    qas['context'] = ptext
                    qas['source'] = url
                    qadict[qas['question']] = qas

                except:
                  1 == 1


                alltextdone = True
                break
          
            else:
              
              #print('len 2', '*'*40 , ptext , '*'*40, len(ptext))
              try:
                listofdict = nlp(ptext)

                for qas in listofdict:
                  #print(qas)
                  qas['context'] = ptext
                  qas['source'] = url

                  qadict[qas['question']] = qas
              except:
                1 == 1
              
              try:
                listofdict = nlp_mtask(ptext)

                for qas in listofdict:
                  qas['context'] = ptext
                  qas['source'] = url
                  qadict[qas['question']] = qas

              except:
                1 == 1
            

              break
            
        else:
          break

https://pytorch.org/docs/stable/generated/torch.bitwise_and.html#torch.bitwise_and


## Single task QA

In [ ]:
text2 = """PyTorch is an optimized tensor library for deep learning using GPUs and CPUs.
Features described in this documentation are classified by release status:
Stable:  These features will be maintained long-term and there should generally
be no major performance limitations or gaps in documentation.
We also expect to maintain backwards compatibility (although
breaking changes can happen and notice will be given one release ahead
of time).
Beta:  These features are tagged as Beta because the API may change based on
user feedback, because the performance needs to improve, or because
coverage across operators is not yet complete. For Beta features, we are
committing to seeing the feature through to the Stable classification.
We are not, however, committing to backwards compatibility.
Prototype:  These features are typically not available as part of
binary distributions like PyPI or Conda, except sometimes behind run-time
flags, and are at an early stage for feedback and testing.
Notes
Language Bindings
Python API
Libraries
Community
Index
Module Inde"""

In [ ]:
%cd question_generation

/content/question_generation


In [ ]:
from pipelines import pipeline

In [ ]:
nlp = pipeline("question-generation")

In [ ]:
#nlp(text3)

If you want to use the t5-base model, then pass the path through model parameter

In [ ]:
nlp = pipeline("question-generation", model="valhalla/t5-base-qg-hl")


In [ ]:
nlp_mtask = pipeline("multitask-qa-qg", model="valhalla/t5-base-qa-qg-hl")


In [ ]:
txt = """Matrix product of two tensors.The behavior depends on the dimensionality of the tensors as follows:If both tensors are 1-dimensional, the dot product (scalar) is returned.If both arguments are 2-dimensional, the matrix-matrix product is returned.If the first argument is 1-dimensional and the second argument is 2-dimensional,
a 1 is prepended to its dimension for the purpose of the matrix multiply.
After the matrix multiply, the prepended dimension is removed.If the first argument is 2-dimensional and the second argument is 1-dimensional,
the matrix-vector product is returned.If both arguments are at least 1-dimensional and at least one argument is
N-dimensional (where N > 2), then a batched matrix multiply is returned.  If the first
argument is 1-dimensional, a 1 is prepended to its dimension for the purpose of the
batched matrix multiply and removed after.  If the second argument is 1-dimensional, a
1 is appended to its dimension for the purpose of the batched matrix multiple and removed after.The non-matrix (i.e. )"""

"""# is appended to its dimension for the purpose of the batched matrix multiple and removed after.
The non-matrix (i.e. batch) dimensions arebroadcasted(and thus
must be broadcastable)."""

nlp(txt)

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


[{'answer': 'Matrix product of two tensors',
  'question': 'What is the behavior dependent on the dimensionality of the tensors?'},
 {'answer': 'the matrix multiply',
  'question': 'When is the prepended dimension removed?'},
 {'answer': 'batched matrix multiply',
  'question': 'If the first argument is 1-dimensional, a 1 is prepended to its dimension for the purpose of what?'},
 {'answer': 'non-matrix',
  'question': 'What is another term for a matrix product of two tensors?'}]

In [ ]:
len(txt)

1032

### 512 length

In [ ]:
uniqdict = {}
qadict = {}

#nlp = pipeline("question-generation", model="valhalla/t5-base-qg-hl")
lengthlist = [512 , 1024, 2048 , 3072 , 4096]


for url in teamurllist[start_idx:end_idx]:
  #url = 'https://pytorch.org/docs/stable/tensor_attributes.html#type-promotion-doc' 
  # 'https://pytorch.org/docs/stable/tensor_attributes.html#type-promotion-doc'
  #'https://pytorch.org/docs/stable/autograd.html'
  # 'https://pytorch.org/docs/stable/generated/torch.load.html#torch.load'
  #print(url)
  try:
    html = req.urlopen(url).read()
    #print(html)
  except:
    continue
  soup = BeautifulSoup(html)




  article = soup.find('article')
  
  if article:
    

    sctns = article.find_all('div', class_="section")

    for sctn in sctns:
      ptags = sctn.find_all('p')

      
      ptextlist = []

      i = 1
      for ptag in ptags:
        ptextlist.append([ i , ptag.get_text(strip=True)])
        i += 1

      #print(ptextlist)

      ptextlistlen = len(ptextlist)
      print(ptextlistlen)

      ptextlt1024list = []
      alltextdone = False

      for j in range(40):
        ptext = ""
        if not alltextdone:

          for k in range(j,ptextlistlen):
          
            arry = ptextlist[k]
            #print(arry)
            if len(ptext) + len(arry[1]) + 1 < 512: #1024:
              ptext += arry[1]
              ptext += ' '

              if arry[0] == ptextlistlen:

                #print('len 1',len(ptext))
                try:
                  listofdict = nlp(ptext)

                  for qas in listofdict:
                    qas['context'] = ptext
                    qas['source'] = url
                    qadict[qas['question']] = qas

                except:
                  1 == 1
                
                
                try:
                  listofdict = nlp_mtask(ptext)

                  for qas in listofdict:
                    qas['context'] = ptext
                    qas['source'] = url
                    qadict[qas['question']] = qas

                except:
                  1 == 1


                alltextdone = True
                break
          
            else:
              
              #print('len 2', '*'*40 , ptext , '*'*40, len(ptext))
              try:
                listofdict = nlp(ptext)

                for qas in listofdict:
                  #print(qas)
                  qas['context'] = ptext
                  qas['source'] = url

                  qadict[qas['question']] = qas
              except:
                1 == 1
              
              try:
                listofdict = nlp_mtask(ptext)

                for qas in listofdict:
                  qas['context'] = ptext
                  qas['source'] = url
                  qadict[qas['question']] = qas

              except:
                1 == 1
            

              break
            
        else:
          break
     
          



5


In [ ]:
qas['context']

NameError: ignored

In [ ]:
k = 0
for v in qadict.values():
  print( k , '*'*40,'\n question :-' , v['question'] ,'\n answer:-' , v['answer'] ,'\n context:-' , v['context'] ,'\n source:-' , v['source'])
  k += 1

### 1024 length

In [ ]:
uniqdict = {}
qadict_1024 = {}

#nlp = pipeline("question-generation", model="valhalla/t5-base-qg-hl")
lengthlist = [512 , 1024, 2048 , 3072 , 4096]


for url in teamurllist[start_idx:end_idx]:
  #url = 'https://pytorch.org/docs/stable/tensor_attributes.html#type-promotion-doc' 
  # 'https://pytorch.org/docs/stable/tensor_attributes.html#type-promotion-doc'
  #'https://pytorch.org/docs/stable/autograd.html'
  # 'https://pytorch.org/docs/stable/generated/torch.load.html#torch.load'
  #print(url)
  try:
    html = req.urlopen(url).read()
  except:
    continue
  soup = BeautifulSoup(html)




  article = soup.find('article')
  
  if article:

    sctns = article.find_all('div', class_="section")

    for sctn in sctns:
      ptags = sctn.find_all('p')

      
      ptextlist = []

      i = 1
      for ptag in ptags:
        ptextlist.append([ i , ptag.get_text(strip=True)])
        i += 1

      #print(ptextlist)

      ptextlistlen = len(ptextlist)

      ptextlt1024list = []
      alltextdone = False

      for j in range(40):
        ptext = ""
        if not alltextdone:

          for k in range(j,ptextlistlen):
          
            arry = ptextlist[k]
            #print(arry)
            if len(ptext) + len(arry[1]) + 1 < 1024:
              ptext += arry[1]
              ptext += ' '

              if arry[0] == ptextlistlen:

                #print('len 1',len(ptext))
                try:
                  listofdict = nlp(ptext)

                  for qas in listofdict:
                    qas['context'] = ptext
                    qas['source'] = url
                    qadict_1024[qas['question']] = qas

                except:
                  1 == 1

                try:
                  listofdict = nlp_mtask(ptext)

                  for qas in listofdict:
                    qas['context'] = ptext
                    qas['source'] = url
                    qadict[qas['question']] = qas

                except:
                  1 == 1

              
                alltextdone = True
                break
          
            else:
              
              #print('len 2', '*'*40 , ptext , '*'*40, len(ptext))
              try:
                listofdict = nlp(ptext)

                for qas in listofdict:
                  #print(qas)
                  qas['context'] = ptext
                  qas['source'] = url

                  qadict_1024[qas['question']] = qas
              except:
                1 == 1
              
              try:
                listofdict = nlp_mtask(ptext)

                for qas in listofdict:
                  qas['context'] = ptext
                  qas['source'] = url
                  qadict[qas['question']] = qas

              except:
                1 == 1

            
              break
            
        else:
          break
     
          



### 2048 length

In [ ]:
uniqdict = {}
qadict_2048 = {}

#nlp = pipeline("question-generation", model="valhalla/t5-base-qg-hl")
lengthlist = [512 , 1024, 2048 , 3072 , 4096]


for url in teamurllist[start_idx:end_idx]:
  #url = 'https://pytorch.org/docs/stable/tensor_attributes.html#type-promotion-doc' 
  # 'https://pytorch.org/docs/stable/tensor_attributes.html#type-promotion-doc'
  #'https://pytorch.org/docs/stable/autograd.html'
  # 'https://pytorch.org/docs/stable/generated/torch.load.html#torch.load'
  #print(url)
  try:
    html = req.urlopen(url).read()
  except:
    continue
  soup = BeautifulSoup(html)




  article = soup.find('article')
  
  if article:

    sctns = article.find_all('div', class_="section")

    for sctn in sctns:
      ptags = sctn.find_all('p')

      
      ptextlist = []

      i = 1
      for ptag in ptags:
        ptextlist.append([ i , ptag.get_text(strip=True)])
        i += 1

      #print(ptextlist)

      ptextlistlen = len(ptextlist)

      ptextlt1024list = []
      alltextdone = False

      for j in range(40):
        ptext = ""
        if not alltextdone:

          for k in range(j,ptextlistlen):
          
            arry = ptextlist[k]
            #print(arry)
            if len(ptext) + len(arry[1]) + 1 < 2*1024:
              ptext += arry[1]
              ptext += ' '

              if arry[0] == ptextlistlen:

                #print('len 1',len(ptext))
                try:
                  listofdict = nlp(ptext)

                  for qas in listofdict:
                    qas['context'] = ptext
                    qas['source'] = url
                    qadict_2048[qas['question']] = qas

                except:
                  1 == 1
                
                try:
                  listofdict = nlp_mtask(ptext)

                  for qas in listofdict:
                    qas['context'] = ptext
                    qas['source'] = url
                    qadict[qas['question']] = qas

                except:
                  1 == 1

              
                alltextdone = True
                break
          
            else:
              
              #print('len 2', '*'*40 , ptext , '*'*40, len(ptext))
              try:
                listofdict = nlp(ptext)

                for qas in listofdict:
                  #print(qas)
                  qas['context'] = ptext
                  qas['source'] = url

                  qadict_2048[qas['question']] = qas
              except:
                1 == 1
              try:
                listofdict = nlp_mtask(ptext)

                for qas in listofdict:
                  qas['context'] = ptext
                  qas['source'] = url
                  qadict[qas['question']] = qas

              except:
                1 == 1
              
            
              break
            
        else:
          break
     
          



In [ ]:

k = 0
for v in qadict.values():
  print( k , '*'*40,'\n question :-' , v['question'] ,'\n answer:-' , v['answer'] ,'\n context:-' , v['context'] ,'\n source:-' , v['source'])
  k += 1

In [ ]:
k = 0
for v in qadict_1024.values():
  print( k , '*'*40,'\n question :-' , v['question'] ,'\n answer:-' , v['answer'] ,'\n context:-' , v['context'] ,'\n source:-' , v['source'])
  k += 1

In [ ]:

k = 0
for v in qadict_2048.values():
  print( k , '*'*40,'\n question :-' , v['question'] ,'\n answer:-' , v['answer'] ,'\n context:-' , v['context'] ,'\n source:-' , v['source'])
  k += 1

In [ ]:
combkeys = list(qadict.keys()) + list(set(qadict.keys()) - set(qadict_1024.keys()))

combkeys123 = combkeys + list(set(combkeys) - set(qadict_2048.keys()))

len(combkeys123) , len(combkeys) , len(qadict.keys()), len(qadict_1024.keys()) , len(qadict_2048.keys())

(218, 149, 83, 21, 24)

In [ ]:
onelist = {}
for key in combkeys123:
  if key in qadict_2048:
    onelist[key] = qadict_2048[key]
  elif key in qadict_1024:
    onelist[key] = qadict_1024[key]
  elif key in qadict:
    onelist[key] = qadict[key]

len(onelist)




83

In [ ]:

with open('/content/onlytextpytorch.json', 'w') as file:
  file.write(json.dumps(onelist, indent=4))

## Multitask QA-QG

### small-model

In [ ]:
nlp2 = pipeline("multitask-qa-qg")

#### QG

nlp(text)

In [ ]:
nlp2(text2)

In [ ]:
nlp2(text4)

#### QA

In [ ]:
nlp2({
  "question": "Who created Python ?",
  "context": text
})

In [ ]:
nlp2({
    "question": "Who wrote Forrest Gump ?",
     "context": text4
})

### base-model

#### QG

In [ ]:
nlp2(text2)

#### QA

In [ ]:
"""nlp({
    "question": "Who wrote Forrest Gump ?",
     "context": text4
})"""

## End-to-End QG

### small model

In [ ]:
nlp3 = pipeline("e2e-qg")

In [ ]:
nlp3(text2)

### base-model

In [ ]:
nlp4 = pipeline("e2e-qg", model="valhalla/t5-base-e2e-qg")

In [ ]:
s 

In [ ]:
nlp4(text2)